In [1]:
import numpy as np
from pc_kriging import PC_Kriging

## Reference function

One dimension

In [2]:
def sin_invented(x):
    dimension = x.shape[1]
    datapoints = x.shape[1]
    result = np.zeros((datapoints, dimension))
    result = 3*np.sin(x[:, 0])
    return result

Two dimensions

In [3]:
def gfun_53(x):
    """Performance function for reliability problem 53.

    Parameters
    ----------
        x : numpy.array of float(s)
            Values of independent variables: columns are the different parameters/random variables (x1, x2,...xn) and rows are different parameter/random variables sets for different calls.

    Returns
    -------
        g_val_sys : numpy.array of float(s)
            Performance function value for the system.
        g_val_comp : numpy.array of float(s)
            Performance function value for each component.
        msg : str
            Accompanying diagnostic message, e.g. warning.
    """
#     import numpy as np
    # expected number of random variables/columns
    nrv_e = 2

    g = float('nan')
    msg = 'Ok'
    x = np.array(x, dtype='f')

    n_dim = len(x.shape)
    if n_dim == 1:
        x = np.array(x)[np.newaxis]
    elif n_dim > 2:
        msg = 'Only available for 1D and 2D arrays.'
        return float('nan'), float('nan'), msg

    nrv_p = x.shape[1]
    if nrv_p != nrv_e:
        msg = f'The number of random variables (x, columns) is expected to be {nrv_e} but {nrv_p} is provided!'
    else:
        g = np.sin(5*x[:, 0]/2) + 2 - (x[:, 0]**2 + 4)*(x[:, 1] - 1)/20

    g_val_sys = g
    g_val_comp = g
    return g_val_sys, g_val_comp, msg

### Creating object

In [3]:
config = {"pol_type": ['hermite']}

In [4]:
config = {"pol_type": ['hermite', 'hermite']}

In [4]:
krig = PC_Kriging(config)

In [ ]:
krig = PC_Kriging()

### Creating and evaluating ref function

Two dimensions

In [6]:
function = gfun_53   #choose the ground truth function

dim = 2       # dimensionality
n = 40         # observations
seed = 0      # fixing seed random number generator

np.random.seed(seed)         #fixing seed random number generator

xn = np.zeros((int(n), dim))      #normalized training points
xr = np.zeros((int(n), dim))      #scaled training points

#-----------------------------------------------------random Variable 1
x1mean = 1.5                           
x1sigma = 1.0

xn[:,0] = np.random.normal(0, 1, n)                             
xr[:,0] = krig.scalehermite(xn[:,0], x1mean, x1sigma) 
 
#-----------------------------------------------------random Variable 2

x2mean = 2.5              
x2sigma = 1.0  

xn[:,1] = np.random.normal(0, 1, n)                             
xr[:,1] = krig.scalehermite(xn[:,1], x2mean, x2sigma) 

#=====================================================
# Evaluation of observations 
y = function(xr)          #Observations
y = y[0]

One dimension

In [5]:
function = sin_invented   #choose the ground truth function

dim = 1       # dimensionality
n = 5         # observations
seed = 0      # fixing seed random number generator

np.random.seed(seed)         #fixing seed random number generator

xn = np.zeros((int(n), dim))      #normalized training points
xr = np.zeros((int(n), dim))      #scaled training points

#-----------------------------------------------------random Variable 1
x1mean = 1.5                           
x1sigma = 1.0

xn[:,0] = np.random.normal(0, 1, n)                             
xr[:,0] = krig.scalehermite(xn[:,0], x1mean, x1sigma) 

#=====================================================
# Evaluation of observations 
y = function(xr)          #Observations
# y = y[0]

## Training the model

In [6]:
y

array([-0.36646154,  2.83874776,  1.84610855, -1.69219515, -0.67214179])

In [7]:
#train model -----------------------------------------

# kernel hyperparameters------------------------------
l = 1.21   #length scale
v = 5/2    #matern coefficient
theta = np.array([l,v])

#truncation term-------------------------------------
p = 2  #for each variable → same truncation , degree of expansion

modelpar1 = krig.train(xn, y, p, theta)    # returns B, sig2, InfoMatrix(phi) , PolyIndices(alpha)
modelpar1[0], modelpar1[1]   ## B, sig2,

(array([ 2.9336282 , -1.34599544, -0.49265239]), 0.11419275619841965)

### Generating prediction points

In [ ]:
# model predictions -----------------------------------------

ntest = 20  # test points
seed = 0      # fixing seed random number generator
np.random.seed(seed)         #fixing seed random number generator

# test points -------------------------------------------------
XR = np.zeros((int(ntest), dim))   #normalized test points
XN = np.zeros((int(ntest), dim))  #scaled test points
#variable 1 ---------------------------------------------------
XN[:,0] = np.random.normal(0,1,ntest)  
XR[:,0] = krig.scalehermite(XN[:,0],x1mean,x1sigma)  
#variable 2 ---------------------------------------------------
XN[:,1] = np.random.normal(0,1,ntest)  
XR[:,1] = krig.scalehermite(XN[:,1],x2mean,x2sigma)  

# mean1,var1 = PCK_predict(XN,xn,y,theta,modelpar1)

### Predicting

In [ ]:
#predictions over a grid (for plotting)--------------------------------------------------

X1t, X2t = np.meshgrid(np.sort(XN[:,0]), np.sort(XN[:,1])) #grid over normalized points
Xt = np.stack((X1t,X2t),axis=2).reshape(-1,2) #sorting the random sampling (graphing purpose)

X1m, X2m = np.meshgrid(np.sort(XR[:,0]), np.sort(XR[:,1])) #grid over non-normalized points
Xtm = np.stack((X1m,X2m),axis=2).reshape(-1,2) #sorting the random sampling (graphing purpose)

yt = function(Xtm)    #test points
Mtest = yt[0]

response1 = krig.predict(Xt, xn, y, theta, modelpar1)   #model predictions over (ntest x ntest) points
mean1 = response1[0]
var1 = response1[1]

In [ ]:
mean1